In [ ]:
!pip install beautifulsoup4
!pip install pandas
!pip install requests

In [1]:
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
# Function to write data to CSV file

def write_to_csv(data, filename):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write header row
        writer.writerow(['title', 'link', 'date', 'content', 'is_fake', 'media_bias'])
        # Write data rows
        for row in data:
            writer.writerow(row)

In [3]:
# KOMPAS.COM News Scraping for HOAX News

url = 'https://www.kompas.com/cekfakta/hoaks-atau-fakta'

# Initialize list to store data
data = []

for i in range(1, 201):
  url_page = url+'/{}'.format(i)
  page = requests.get(url_page)
  soup = BeautifulSoup(page.text, 'html')

  # Find all news articles
  news = soup.find_all('div', 'col-bs9-3')

  for n in news:
    title_category = n.find('h1', 'cekfakta-list-title').get_text()
    link = n.find('a', 'cekfakta-list-link')['href']
    date_str = n.find('p', 'cekfakta-text-date').get_text().split(',')
    date = datetime.strptime(date_str[0].strip(), '%d/%m/%Y').date()

    # Split title and category
    part_title = title_category.split(']')
    if len(part_title) == 2:
      category = part_title[0].strip('[').strip()
      title = part_title[1].strip()

      if category == "HOAKS" and link is not None:
        # Get news contents
        get_content = requests.get(link)
        soup_content = BeautifulSoup(get_content.text, 'html')
        content = soup_content.find('div', 'read__content').find_all('p')
        paragraph = ''
        for p in content:
          paragraph += p.get_text()

        # News information source
        is_fake = 1
        media_bias = 'netral'

        # Append data to list
        data.append([title, link, date, paragraph, is_fake, media_bias])

# File name for the CSV
filename = 'dataset_kompas_hoaks.csv'

# Write data to CSV file
write_to_csv(data, filename)

print(f"Data has been written to {filename}")

Data has been written to dataset_kompas_hoaks.csv


In [4]:
# Read CSV file
df = pd.read_csv('/content/dataset_kompas_hoaks.csv')
df.head()

,title,link,date,content,is_fake,media_bias
0,Harimau Mati Tertabrak Kendaraan di Tol Pekanb...,https://www.kompas.com/cekfakta/read/2024/05/1...,2024-05-18,"Berdasarkan verifikasi Kompas.com sejauh ini, ...",1,netral
1,Mantan PM Jepang Dibunuh karena Tidak Patuh pa...,https://www.kompas.com/cekfakta/read/2024/05/1...,2024-05-18,"Berdasarkan verifikasi Kompas.com sejauh ini, ...",1,netral
2,Prabowo-Gibran Batal Dilantik oleh MPR,https://www.kompas.com/cekfakta/read/2024/05/1...,2024-05-18,"Berdasarkan verifikasi Kompas.com sejauh ini, ...",1,netral
3,Foto Ular Raksasa di Carolina Selatan,https://www.kompas.com/cekfakta/read/2024/05/1...,2024-05-17,"Berdasarkan verifikasi Kompas.com sejauh ini, ...",1,netral
4,Warga Rafah Bikin Video Rekayasa Serangan Israel,https://www.kompas.com/cekfakta/read/2024/05/1...,2024-05-17,"Berdasarkan verifikasi Kompas.com sejauh ini, ...",1,netral


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       1757 non-null   object
 1   link        1757 non-null   object
 2   date        1757 non-null   object
 3   content     1757 non-null   object
 4   is_fake     1757 non-null   object
 5   media_bias  1757 non-null   object
dtypes: object(6)
memory usage: 82.5+ KB


In [6]:
# Save dataset to Google Drive

from google.colab import drive
import shutil

drive.mount('/content/drive')

colab_file_path = '/content/dataset_kompas_hoaks.csv'
drive_file_path = '/content/drive/My Drive/Colab Notebooks/Bangkit | PukulRata/dataset_kompas_hoaks.csv'

shutil.copyfile(colab_file_path, drive_file_path)

Mounted at /content/drive


'/content/drive/My Drive/Colab Notebooks/Bangkit | PukulRata/dataset_kompas_hoaks.csv'